# Point Cloud Modeling using Self-Organizing Gaussian Mixture Models (SOGMM)
This tutorial presents modeling a point cloud using the SOGMM approach presented in [1].
We assume that a depth-intensity point cloud with $N$ points (i.e., point cloud is a set of
$N$ four-dimensional points $\mathbf{x}$) is given. The output is a Gaussian Mixture Model (GMM)
of the point cloud that can be used for:
1. Inference for intensity image reconstruction
2. Dense sampling for 3D point cloud reconstruction 

## Input
For the purposes of this tutorial, download the point cloud data from
[this](https://cmu.box.com/s/9v7v1knt9iswui2518c6qpqtzdqnf2v5) link.  The
downloaded zip folder contains point cloud data from a few frames of the
`stonewall`, `copyroom`, and `lounge` ICL-NUIM datasets [2].

Extract the zip folder. You will get a folder with files named like this:

```
gira3d-tutorial-data
├── pcd_copyroom_2364_decimate_1_0.pcd
├── pcd_copyroom_2364_decimate_2_0.pcd
└── ...
```
The files are named as `pcd_<dataset-name>_<frame-number>_decimate_<decimation_amount>.pcd`.
Here, decimation of 1.0 means the original image of size `640 x 480` was used to generate the point cloud.
Likewise, decimations of `2.0`, `3.0`, and `4.0` correspond to image sizes `320 x 240`, `213 x 160`, and
`160 x 120` respectively.

For the purposes of this tutorial, we limit to the frames provided in the folder above. However, the
methodology will work on any registered depth-intensity image pair.

## Learning the SOGMM Model
We start by importing the run time dependencies:

In [1]:
%matplotlib inline
import numpy as np
import open3d as o3d

from sogmm_py.utils import read_log_trajectory, o3d_to_np, np_to_o3d
from sogmm_py.vis_open3d import VisOpen3D

For the purposes of this tutorial, we will use frame `1763` of the `lounge` dataset.

In [58]:
# frame = 935
# datasetname = 'copyroom'

# frame = 103
# datasetname = 'stonewall'

frame = 1763
datasetname = 'lounge'

Import a ground truth point cloud from the dataset folder.

In [60]:
pcld_gt = o3d.io.read_point_cloud('./gira3d-tutorial-data/pcd_' +
                                  str(datasetname) +
                                  '_' + str(frame) +
                                  '_decimate_1.pcd', format='pcd')
pcld_gt_np = o3d_to_np(pcld_gt)

In [66]:
pcld_gt_np.shape

(216035, 4)

Import the ground truth SE(3) pose.

In [61]:
traj = read_log_trajectory('./gira3d-tutorial-data/' +
                           str(datasetname) + '-traj.log')
pcld_pose = traj[frame].pose

Initialize the intrinsics matrix and width and height variables.

In [62]:
K = np.eye(3)
K[0, 0] = 525.0
K[1, 1] = 525.0
K[0, 2] = 319.5
K[1, 2] = 239.5

W = (int)(640)
H = (int)(480)

To visualize, please use our python wrapper over the Open3D visualizer.

In [63]:
vis = VisOpen3D(visible=True)
vis.visualize_pcld(pcld_gt, pcld_pose, K, W, H)
vis.render()
del vis

You should see an interactive Open3D window that shows the depth-intensity point cloud and the camera frustrum:
![Ground Truth Point Cloud](results/sogmm-tut-1.png)

Since the SOGMM method produces a generative model of the point cloud, we can train it on decimated point clouds. In this example, let us use a decimation of `4.0`.

In [54]:
deci = 6.0

Get the decimated point cloud from the dataset folder:

In [55]:
pcld_deci = o3d.io.read_point_cloud('./gira3d-tutorial-data/pcd_' +
                                  str(datasetname) +
                                  '_' + str(frame) +
                                  '_decimate_' +
                                  str(deci).replace('.', '_') +
                                  '.pcd', format='pcd')
pcld_deci_np = o3d_to_np(pcld_deci)

Initialize the intrinsics matrix and width and height variables corresponding to the decimation factor.

In [56]:
K_d = np.eye(3)
K_d[0, 0] = 525.0/deci
K_d[1, 1] = 525.0/deci
K_d[0, 2] = 319.5/deci
K_d[1, 2] = 239.5/deci

W_d = (int)(640/deci)
H_d = (int)(480/deci)

Visualize the decimated point cloud the same way.

In [57]:
vis = VisOpen3D(visible=True)
vis.visualize_pcld(pcld_deci, pcld_pose, K_d, W_d, H_d)
vis.render()
del vis

The decimated point cloud looks like this:
![Decimated Point Cloud](results/sogmm-tut-2.png)

Next, we learn a SOGMM on the decimated point cloud. Both CPU-only and GPU-accelerated cases are supported. Both of these modules can be accessed through the `SOGMM` class.

In [11]:
from sogmm_py.sogmm import SOGMM

### SOGMM CPU

In [12]:
sg_cpu = SOGMM(bandwidth=0.02, compute='CPU')
model_cpu = sg_cpu.fit(pcld_deci_np)

Compute platform is CPU


We can take a look at the number of components automatically chosen by SOGMM for this frame:

In [13]:
def visualize_gaussians_with_camera(model, camera_pose, K, W, H, scale=2.0):
    """
    Visualize Gaussians with camera position and frustum
    
    Parameters:
    - model: The SOGMM model
    - camera_pose: 4x4 SE(3) transformation matrix for camera position
    - K: 3x3 Camera intrinsics matrix
    - W: Image width
    - H: Image height 
    - scale: Base scale factor for ellipsoids
    """
    import open3d as o3d
    import numpy as np
    
    # Create visualization
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    
    # Add camera frustum visualization
    # Create camera visualization using K, W, H
    # Using camera intrinsics to create frustum
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    
    # Create a simple camera frustum
    camera_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
    
    # Create frustum lines
    points = []
    lines = []
    colors = []
    
    # Camera center
    camera_center = np.array([0, 0, 0])
    points.append(camera_center)
    
    # Define the corners of the image plane at z=1
    z = 1.0
    top_left = np.array([(0 - cx) * z / fx, (0 - cy) * z / fy, z])
    top_right = np.array([(W - cx) * z / fx, (0 - cy) * z / fy, z])
    bottom_left = np.array([(0 - cx) * z / fx, (H - cy) * z / fy, z])
    bottom_right = np.array([(W - cx) * z / fx, (H - cy) * z / fy, z])
    
    # Scale frustum for better visibility
    frustum_scale = 0.3
    top_left *= frustum_scale
    top_right *= frustum_scale
    bottom_left *= frustum_scale
    bottom_right *= frustum_scale
    
    # Add corners
    points.extend([top_left, top_right, bottom_left, bottom_right])
    
    # Add lines from center to corners
    lines.extend([[0, 1], [0, 2], [0, 3], [0, 4]])
    
    # Add lines around the image plane
    lines.extend([[1, 2], [2, 4], [4, 3], [3, 1]])
    
    # Set a distinct color for frustum
    frustum_color = [1, 0.5, 0]  # Orange
    colors = [frustum_color for _ in range(len(lines))]
    
    # Create line set for frustum
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)
    
    # Transform the camera according to pose
    line_set.transform(camera_pose)
    camera_mesh.transform(camera_pose)
    
    vis.add_geometry(line_set)
    vis.add_geometry(camera_mesh)
    
    # For each Gaussian component
    for i in range(len(model.means_)):
        mean = model.means_[i, :3]
        intensity = model.means_[i, 3]
        
        try:
            # Reshape to 4x4 and extract 3x3 spatial part
            cov_flat = model.covariances_[i]
            cov_4x4 = cov_flat.reshape(4, 4)
            cov = cov_4x4[:3, :3]
            
            # Create ellipsoid
            mesh = o3d.geometry.TriangleMesh.create_sphere()
            
            # Calculate eigenvalues and eigenvectors
            eigenvalues, eigenvectors = np.linalg.eigh(cov)
            
            # Scale by eigenvalues
            component_scale = scale
            
            transform = np.eye(4)
            transform[:3, :3] = np.dot(eigenvectors, np.diag(np.sqrt(np.abs(eigenvalues)) * component_scale))
            transform[:3, 3] = mean
            mesh.transform(transform)
            
            # Set color based on intensity
            gray_val = intensity  # Already in 0-1 range
            mesh.paint_uniform_color([gray_val, gray_val, gray_val])
            
            vis.add_geometry(mesh)
            
        except (np.linalg.LinAlgError, ValueError) as e:
            print(f"Error with component {i}: {e}")
            continue
    
    # Set rendering options
    render_option = vis.get_render_option()
    render_option.background_color = np.array([0.1, 0.1, 0.1])  # Dark background
    render_option.light_on = True
    
    # Set the viewpoint to the camera position
    view_control = vis.get_view_control()
    camera_position = camera_pose[:3, 3]
    # Look camera position minus the camera direction
    view_control.set_lookat(camera_position + 1.5*camera_pose[:3, 2])  # Z-axis of the camera
    view_control.set_up(-camera_pose[:3, 1])  # Y-axis of the camera
    view_control.set_front(-camera_pose[:3, 2])  # Negative Z-axis of the camera
    
    vis.run()
    vis.destroy_window()


visualize_gaussians_with_camera(model_cpu, pcld_pose, K_d, W_d, H_d)

In [14]:
model_cpu.n_components_

588

### SOGMM GPU

In [15]:
sg_gpu = SOGMM(bandwidth=0.02, compute='GPU')
model_gpu = sg_gpu.fit(pcld_deci_np)

Compute platform is GPU


In [16]:
visualize_gaussians_with_camera(model_gpu, pcld_pose, K_d, W_d, H_d)

Let us make sure we got the same number of components.

In [17]:
model_gpu.n_components_

588

## Inference for Intensity Image
Intensity image can be constructed using the conditional GMM as follows. For more details on how the conditional distribution is computed, please refer to [1].

In [18]:
%%time
_, expected_intensities, _ = model_gpu.color_conditional(pcld_gt_np[:, 0:3])

CPU times: user 45.9 s, sys: 850 ms, total: 46.7 s
Wall time: 7.55 s


`expected_intensities` contains the expected intensity values at the ground truth 3D points. Let us visualize these reconstructed intensity values.

In [19]:
recon_pcld = np.zeros(pcld_gt_np.shape)
recon_pcld[:, 0:3] = pcld_gt_np[:, 0:3] # we are constructing intensity image on gt 3D points
recon_pcld[:, 3] = np.squeeze(expected_intensities)

vis = VisOpen3D(visible=True)
vis.visualize_pcld(np_to_o3d(recon_pcld), pcld_pose, K, W, H)
vis.render()
del vis

![Reconstructed Point Cloud](results/sogmm-tut-3.png)

Higher accuracy can be achieved if the learning is performed at a lower `deci` value (at the cost of higher computation time).

## Dense sampling for 3D point cloud reconstruction
Dense sampling from the 4D GMM can be performed using the usual Box-Mueller sampling method [3].

In [20]:
resampled_pcld = sg_gpu.joint_dist_sample(pcld_gt_np.shape[0]) # sample 4D points from the model
vis = VisOpen3D(visible=True)
vis.visualize_pcld(np_to_o3d(resampled_pcld), pcld_pose, K, W, H)
vis.render()
del vis

![Resampled Point Cloud](results/sogmm-tut-4.png)

## Performance Measures
We support computing the following:
1. Peak Signal-To-Noise Ratio (PSNR): To measure the accuracy of intensity image reconstruction.
2. Structure Similarity Index Measure (SSIM): To measure the accuracy of intensity image reconstruction.
3. F-score: To measure the accuracy of resampled point cloud.
4. Precision: To measure the accuracy of resampled point cloud.
5. Recall: To measure the accuracy of resampled point cloud.
6. Mean Reconstruction Error (MRE): To measure the accuracy of resampled point cloud.
7. Std. Dev. Reconstruction Error: To measure the accuracy of resampled point cloud.
8. Memory: To measure the compactness.

In [21]:
from sogmm_py.utils import calculate_depth_metrics, calculate_color_metrics
fsc, pre, re, rm, rs = calculate_depth_metrics(pcld_gt, np_to_o3d(resampled_pcld))
print("fscore %f precision %f recall %f recon. mean %f recon. std. dev. %f" % (fsc, pre, re, rm, rs))

from sogmm_py.utils import ImageUtils
iu = ImageUtils(K) # image manipulation utility
_, gt_g = iu.pcld_wf_to_imgs(pcld_pose, pcld_gt_np) # project gt pcld on camera
if np.isnan(gt_g).any():
    gt_g = np.nan_to_num(gt_g)
_, pr_g = iu.pcld_wf_to_imgs(pcld_pose, recon_pcld) # project recon pcld on camera
if np.isnan(pr_g).any():
    pr_g = np.nan_to_num(pr_g)
psnr, ssim = calculate_color_metrics(gt_g, pr_g) # compare the intensity images
print("psnr %f ssim %f" % (psnr, ssim))

fscore 0.979307 precision 0.993742 recall 0.965286 recon. mean 0.003423 recon. std. dev. 0.003788
psnr 32.009320 ssim 0.825107


In [22]:
# computing memory usage
M = model_gpu.n_components_
mem_bytes = 4 * M * (1 + 10 + 4)
print('memory %d bytes' % (mem_bytes))

memory 35280 bytes


## References
[1] K. Goel, N. Michael, and W. Tabib, “Probabilistic Point Cloud Modeling via Self-Organizing Gaussian Mixture Models,” IEEE Robotics and Automation Letters, vol. 8, no. 5, pp. 2526–2533, May 2023, doi: 10.1109/LRA.2023.3256923.

[2] Q.-Y. Zhou and V. Koltun, “Dense scene reconstruction with points of interest,” ACM Trans. Graph., vol. 32, no. 4, pp. 1–8, Jul. 2013, doi: 10.1145/2461912.2461919.

[3] C. M. Bishop and N. M. Nasrabadi, Pattern recognition and machine
learning. Springer, 2006, vol. 4, no. 4